<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
import tensorflow as tf
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split

In [3]:
print(tf.__version__)

2.5.0


#Question 1

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=100,kernel_size=(3,3), strides=2, input_shape=(720, 1280, 3), padding="SAME"))
model.add(keras.layers.Conv2D(filters=200,kernel_size=(3,3), strides=2, padding="SAME"))
model.add(keras.layers.Conv2D(filters=400,kernel_size=(3,3), strides=2, padding="SAME"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 360, 640, 100)     2800      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 320, 200)     180200    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 160, 400)      720400    
Total params: 903,400
Trainable params: 903,400
Non-trainable params: 0
_________________________________________________________________


#Subquestion: 1
This model has 903400 parameters. 
#Subquestion: 2
Since each parameter is stored in 4 Byte float, so the toal RAM required to store all the parameters would be 903400 X 4 = 3.44 MB RAM.
#Subquestion: 3
Since each parameter is stored in 8 bits(1 Byte), so the total RAM required to store all the parameters would be 0.86 MB.
#Subquestion: 4
Since the weights are only stored once for the entire network, training on a mini batch of 20 images will not change the RAM required to store the parameters. 

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
X_train_full = np.expand_dims(X_train_full, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [7]:
X_train_full = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), X_train_full, dtype=tf.float32).numpy()

In [8]:
X_test = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), X_test, dtype=tf.float32).numpy()

In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42, stratify=y_train_full)

In [10]:
def step_decay2(epoch, lr):
     return lr * tf.math.exp(-0.1)

lrate2 = tf.keras.callbacks.LearningRateScheduler(step_decay2)

In [15]:
earlystopping_cb = keras.callbacks.EarlyStopping (patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint ("mnist_fashion.model.h5" , save_best_only=True,monitor='val_accuracy')

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu', padding="SAME"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding="SAME"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=2))
model.add(keras.layers.Dropout(rate=0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(model.summary())

datagen = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, rotation_range=10, fill_mode='nearest', zoom_range=0.2, shear_range=0.2)
it = datagen.flow(x_train, y_train)
model.fit(it, epochs=100, batch_size=128, validation_data=(x_valid, y_valid), verbose=2, callbacks=[checkpoint_cb, earlystopping_cb, lrate2])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_12 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 12544)            